# 02 Generate and tag sentences


## Script A02

This script generates the training dataset, containing questions and the   corresponding tags (such as granularity and datasets) in the Spacy-format so it can be used for the training.

It loads a csv-file with pre-written questions that contain brackets to insert elements such as {dataset} or {year} or {granularity} etc.  `(see input/question_generator.csv)` Currently, we only have seven questions, please feel free to add more. For every dataset we have, several questions are being generated randomly.

A lot could be improved in this script. At the moment, it only takes data of the so called "Gemeindeportraet" of the Statistics Office of the Canton of Zurich. This is because those data variables are in the same format and use the same variable names. We want of course to get away from this structure and use all kind of statistical OGD.

## Thoughts for improvement

- Include other useful **custom tags**
- **Insert important elements** correctly, such as year data, locality level, etc.
- Improve **generating the questions**: i.e. by insterting many more ways of saying 'take the three first elements', 'take all elements larger than X', etc. etc.
- Optional: Try and **tag every single** word in the datasets as dataset element to improve recognition (and then later use this to attribute them to the right dataset) <br><br>

- Outlook for later: Try and **combine this script with the generation of training data** so it can be used for an approach with seq2SQL or similar


In [3]:
from __future__ import print_function, unicode_literals
import sys
import os
import pandas as pd
import numpy as np
from random import sample
import io, csv
import re
import random
import json
random.seed(12345)

In [4]:
OUT_DIR = "output"
if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)
DATA_DIR = "data"
INPUT_DIR = "input"

In [5]:
def _uniform_cleaning(str_in):
    str_in=str_in.replace("-","")
    str_in=str_in.replace("(","")
    str_in=str_in.replace(")","")
    #str_in=str_in.replace("ü","ue")
    #str_in=str_in.replace("ä","ae")
    #str_in=str_in.replace("ö","oe")
    return(str_in)

In [6]:
def _add_data_type_column(df):
    # create extra column from dataset title
    # where data type is specified
    df['question_type'] = df['dataset_title'].str.extract(r"\[(.*?)\]", expand=False)
    df['question_type'] = np.where(df['question_type'] == '%', "percent", "cardinal")
    df['dataset_title'] = df['dataset_title'].str.replace(r"\[(.*?)\]", "")
    return df

In [7]:
def _sample_locality(data, random_value):
    if random_value == "one locality":
                locality_insert = "in " + sample(list(data['GEBIET_NAME']), 1)[0]
    elif random_value == "one level":
        locality_insert = sample(["für den gesamten Kanton","im Kanton Zürich","auf Bezirksebene",
                                "für alle Bezirke","pro Bezirk","auf Gemeindeebene","für alle Gemeinden","pro Gemeinde"], 1)[0]
    elif random_value == "several localities":
        locality_insert = ""
        local_loop = sample([1,2,3],1)[0]
        for local in range(0,local_loop):
            if local != 0 and local != (local_loop - 1):
                locality_insert += ", "
            if local != 0 and local == (local_loop - 1):
                locality_insert += " und "
            locality_insert += sample(list(data['GEBIET_NAME']), 1)[0]
    return locality_insert

In [8]:
def _sample_time(data, random_value):
    years = data['INDIKATOR_JAHR'].tolist()

    if random_value in ["aktuellste", "neueste"]:
        exact_value = max(years)
        return exact_value, random_value

    elif random_value in ["früheste", "älteste"]:
        exact_value = min(years)
        return exact_value, random_value

    elif random_value == "year":
        return random.choice(years), "in " + str(random.choice(years))

    elif random_value == "span":
        tmp_years = sorted(random.sample(years, 2))
        return tmp_years, "von " + str(tmp_years[0]) + " bis " + str(tmp_years[1])


In [76]:
def _generate_sentences(templates, question_type, dataset_title, retrieval_data, filter_vars): 
    relevant_templates = templates.loc[templates.main_type == question_type]
    #sentences, solution_dicts = list(), list()
    ret, tagged = list(), list()
    for _, column in relevant_templates.iterrows():
        tmp_template = column.question
        orig_template = tmp_template
        solution_dict = dict()
        # replace placeholder with column title extracted above  
        tmp_template = tmp_template.replace("{main}", dataset_title)
        solution_dict["main"] = dataset_title

        tmp_template = tmp_template.replace("{localitylevel}", "")#temporary: empty
        solution_dict["localitylevel"] = ""

        #TODO either one locality, one level, or several localities
        random_loc = sample(["one locality", "one level", "several localities"], 1)[0]
        solution_dict["type_locality"] = random_loc
        locality_insert = _sample_locality(retrieval_data, random_loc)
        tmp_template = tmp_template.replace("{locality}", locality_insert)
        solution_dict["locality_insert"] = locality_insert

        random_year = sample(["year", "span", "aktuellste", "älteste"], 1)[0]#"neueste", "früheste",
        retrieval_time, time_insert = _sample_time(retrieval_data, random_year)
        tmp_template = tmp_template.replace("{yeartime}", time_insert)
        solution_dict["type_year"] = random_year
        solution_dict["yeartime"] = retrieval_time
        solution_dict["stringtime"] = time_insert

        filter_insert = sample(filter_vars, 1)[0]
        tmp_template = tmp_template.replace("{filter}", filter_insert)
        solution_dict["filter"] = filter_insert

        sentence = tmp_template
        """
        for mat in re.findall(r'.*?\[(.*)].*', tmp_template):
            which_part = sample([1,2],1)
            if which_part==1:
                tmp_template = tmp_template.replace("["+mat+"]", mat.partition("|")[0])
            else:
                tmp_template = tmp_template.replace("["+mat+"]", mat.partition("|")[2])
        """
        #now the symbol - has to be deleted as it gives issues
        sentence =  _uniform_cleaning(tmp_template)
        solution_dict["sentence"] = sentence

        tagged_tuple = _tag_generated_sentence(solution_dict)#['sentence'], dataset_title)

        tagged.append(tagged_tuple)
        ret.append(solution_dict)
    return ret, tagged



In [81]:
def _tag_generated_sentence(generated_solution, tag_types={
                    'main': "GRAN",
                    'locality_insert': "PLACE",
                    'stringtime': "TIME",
            }):
    #{'main': 'Eigenkapital', 'localitylevel': '', 'type_locality': 'several localities', 'locality_insert': 'Dänikon und Region Weinland', 'type_year': 'year', 'yeartime': 2006, 'filter': '', 'sentence': 'Wie viel Eigenkapital hat  Dänikon und Region Weinland in 1991 ?'}
    ret = {"entities" : []}
    sentence = generated_solution["sentence"]
    for key, tag in tag_types.items():
        search_for = generated_solution[key]
        match = re.search(search_for, sentence, flags=re.IGNORECASE)
        if match:
            ret["entities"].append([match.start(), match.end(), tag])
    
        
    return [sentence, ret]



In [68]:
def _get_filter_vars(value_types):
    filtered_values = [i for i in value_types if i not in ["INDIKATOR_JAHR", "GEBIET_NAME", "BFS_NR", "INDIKATOR_VALUE"]]
    filtered_values.append("")
    return filtered_values

In [77]:
def generate_training_data(data, templates, outfile_ner, outfile_ir):
    """
    :param pd.Dataframe data: df with string data
    :param pd.Dataframe templates: df containing templates for query generation
    :param string outfile_ner: file to safe training data for NER
    :param string outfile_ir: file to write training data for information retrieval
    """
    tagged_sentences, solutions = list(), list()
    data = _add_data_type_column(data)
    for id_, group in data.groupby('index'):
        fname_retrieval_data = os.path.join(DATA_DIR, str(id_) + ".csv")
        problematic_files = list()
        try:
            with open(fname_retrieval_data, "r", encoding="utf-8") as inf:
                retrieval_data = pd.read_csv(inf, sep=";|:")
        except:#unicodedecode, pd.errors.ParserError:
            problematic_files.append(fname_retrieval_data)
        
        value_types = group['var'].tolist()#todo: rename var column -> reserved vocab in pandas
        dataset_title = group.dataset_title.unique()[0]
        question_type = group.question_type.unique()[0]
        
        # temporary: only take columns containing the main value of the gemeindeportraet dataset----------
        if "INDIKATOR_VALUE" in value_types:
            #title of the column 
            dataset_title = _uniform_cleaning(dataset_title.strip())

            # temporary: because it is standardized
            filter_vars = _get_filter_vars(value_types)
            
            generated_solutions, tagged_tuples = _generate_sentences(templates, question_type, dataset_title, retrieval_data, filter_vars)
            
            tagged_sentences += tagged_tuples
            solutions += generated_solutions

    return tagged_sentences, solutions




In [82]:
# load dataset descriptions and template sentences 
overview_data = pd.read_csv(os.path.join(DATA_DIR, "datasets_overview.csv"))
template_data = pd.read_csv(os.path.join(INPUT_DIR, "question_generator.csv"))


tagged_sentences, solutions = generate_training_data(data=overview_data,
                        templates=template_data,
                        outfile_ner=os.path.join(OUT_DIR, "spacy_training_sentences.json"),
                        outfile_ir=os.path.join(OUT_DIR, "info_retrieval_training_data.csv")
                        )


<ipython-input-6-0ca5468b0b90>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['dataset_title'] = df['dataset_title'].str.replace(r"\[(.*?)\]", "")
<ipython-input-77-6c55996b6cdd>:15: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  retrieval_data = pd.read_csv(inf, sep=";|:")


In [84]:
# save data
with open(os.path.join(INPUT_DIR, "tagged_sentences_latest.json"), "w", encoding='utf-8') as outfile:
    json.dump(tagged_sentences, outfile, ensure_ascii=False)

training_data_ir = pd.DataFrame(solutions)
with open(os.path.join(INPUT_DIR, "info_retrieval_data_latest.csv"), "w") as outf:
    training_data_ir.to_csv(outf)